In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, approx_count_distinct
from pyspark.sql.types import DoubleType, StringType
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("train transaction") \
    .getOrCreate()
sparkContext=spark.sparkContext

In [3]:
train_transaction = spark.read.csv("train_transaction.csv",header=True)
train_identity = spark.read.csv("train_identity.csv", header = True)
train_combine = train_transaction.join(train_identity,['TransactionID'],how ='left') 
count_row = train_combine.count()
missing = train_combine.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_combine.columns])
null_counts = missing.collect()[0].asDict()
to_drop = [k for k, v in null_counts.items() if v > (count_row*0.8)]
#len(to_drop)

In [5]:
train_drop_null = train_combine.drop(*to_drop)

In [6]:
C_columns=[column for column in train_drop_null.columns if column.startswith("C") and len(column)<=3]
D_columns=[column for column in train_drop_null.columns if column.startswith("D") and len(column)<=3]
V_columns=[column for column in train_drop_null.columns if column.startswith("V") and len(column)<=4]

In [7]:
def remove_duplicate_columns(col_NameList, train):
    # cast column data type to double
    for i in col_NameList:
        train = train.withColumn(i, col(i).cast(DoubleType()))
    
    # calculate the correlation of C columns
    drop = []
    saved = []
   
    col_NameLength = len(col_NameList)
    for i in range(0,col_NameLength):
        if (len(saved) == 0):
            saved.append(col_NameList[i])
        else:
            flag = 0
            for t in range(0,len(saved)):
                corr = train.corr(saved[t],col_NameList[i])
                if(corr>=0.95):
                    drop.append(col_NameList[i])
                    flag = 1                
                    break
            if(flag==0):
                saved.append(col_NameList[i]) 
    return drop

In [14]:
def corre(previousColumn, nextColumn, train):
    for i in previousColumn:
        train = train.withColumn(i, col(i).cast(DoubleType()))
    
    for i in nextColumn:
        train = train.withColumn(i, col(i).cast(DoubleType()))
        
    drop = []
    
    for i in nextColumn:
        for t in previousColumn:
            print(train.corr(i,t))
            if train.corr(i, t) >= 0.95:
                drop.append(i)
                break
    return drop

In [16]:
dropC = remove_duplicate_columns(C_columns, train_drop_null) #8 from 14
print(dropC)

['C2', 'C4', 'C6', 'C8', 'C10', 'C11', 'C12', 'C14']


In [17]:
dropD = remove_duplicate_columns(D_columns, train_drop_null) # 1 from 8
print(dropD)

['D2']


In [17]:
print(len(V_columns[161::]))

131


In [8]:
dropV100 = remove_duplicate_columns(V_columns[161:261], train_drop_null) #  from 161 to 261
print(dropV100)

['V193', 'V195', 'V196', 'V197', 'V201', 'V204', 'V211', 'V212', 'V219', 'V222', 'V225', 'V231', 'V232', 'V233', 'V237', 'V241', 'V244', 'V247', 'V249', 'V251', 'V254', 'V256', 'V259', 'V269', 'V272', 'V273', 'V279', 'V280']


In [9]:
dropV = remove_duplicate_columns(V_columns[261::], train_drop_null) #  from 261 to end
print(dropV)

['V292', 'V295', 'V298', 'V304', 'V306', 'V307', 'V316', 'V317', 'V318']


In [12]:
savedV_161_261 =  [item for item in V_columns[161:261] if item not in dropV100]

In [15]:
drop_final31 = corre(savedV_161_261, V_columns[261::], train_drop_null)

-0.003224594029331703
-0.001651515952888161
-0.006185321672725226
-0.005814404820682324
-0.002132394232464834
-0.0035094277953129915
0.0008382690574347737
0.000593891819315706
-0.0001567083211480239
2.7795246512054927e-05
-0.00042194321956745296
0.002275115299339657
0.0010303473505599878
0.001993772871686693
0.0009601887221527893
-0.0002826929782539408
2.675505290759676e-05
-0.00011555075384396753
0.0029057787473338367
0.002640746033821389
0.012176367533755133
0.0029079537551497925
0.006264546574007859
0.0009086218297462617
-0.00014818479986056118
0.0011206189199220804
0.010164100947655743
0.006055542243977815
0.009628199030129597
-0.000870889371664241
-0.00041220705843943395
0.0001021741501840605
-0.0008247936542576471
-0.0007222066069220023
-0.00385069952350693
-0.0016849154792030797
-0.0011349031268801113
-0.0027310552234054495
-0.0013930928356713271
-0.0027434954555450187
-0.007015553480573702
-0.0036903102805909294
-0.0014155531880003572
-0.005394583718798427
-0.001563671145384761

0.10849245187525611
0.6284793196902797
0.4640874509338345
0.8820714295832308
0.18271881393705
0.5082988352973177
0.4004495547277656
0.2196572115507425
0.21736034767030432
-0.0037576386998806216
0.0021757262108465557
0.03732347223874199
0.06697476418243327
0.17843523394679756
-0.0012288112968355561
-0.0013333745071220627
-0.00070893072837522
-0.0010553233384134324
0.525073959866852
0.20880050135668282
0.23134661394490957
0.03968022856331423
0.03900809599749581
-0.008747607283094525
-0.009504658319932928
-0.006986607139486975
-0.011259881207781687
-0.009272319672023677
-0.006204490144264295
-0.022779210242424762
-0.008247745210838804
-0.0022204994929522115
-0.016669805846381183
0.0013146899020044392
0.001691547012470759
0.011462067185251407
0.05359557613900538
0.021402397073914753
0.16780551896579646
0.12707997200299176
0.1687908421173916
0.00728674367426851
0.02487070194465724
0.024531828536842055
-0.0011815192618109776
-0.000616351452847468
0.17706611969803385
0.20255749294844672
0.037

0.12149746730086665
0.03732262234004777
0.05564807245825864
0.053994639199377914
0.07740102826812292
0.2180048118917636
0.15363245497161612
0.05240235591645744
0.053789931078512786
0.05380863250351244
0.09308200571346857
0.0139085328337276
-0.003640540589925751
0.2699656711168888
0.7196294907077567
0.3465774121608289
0.8317839887118975
0.7645937601966092
0.49415533032653136
0.5431667742752261
0.2179202908383656
0.24106465630233875
0.061422264082719495
0.024484393384283164
0.08180165468383872
0.13850866626954447
0.2377762069373435
0.22487888402295897
0.24283309559389815
0.08456052130179757
0.20471528412450932
0.16067713601739658
0.1368023463656637
0.14217556538906992
0.1601954600866153
0.3475965665776502
0.19681835859815136
0.09397655530097976
0.09480868937860668
0.06312627189225785
0.5602889073034744
0.2678525335182733
0.45143313302927895
0.21218642411384087
0.18390997512657667
0.14495147320409618
0.17054845597291957
0.17144775839785328
0.8157419485908766
0.6831179269896515
0.484690388

0.004123875289606398
0.00655072914651141
0.0001805123376940307
0.0014441003286609206
0.003014668006905133
-0.0005210925167522505
0.03676146394513219
0.023706385033505983
0.6229587386327711
0.8626725203489419
0.18269440612853616
0.114163850333198
0.033400186861617984
0.0814553905030028
0.0017349819691815937
0.0026920654065635953
0.007256108919902493
0.0036774146140868403
0.003196016781423954
0.0020946615685262775
0.025744196576584968
0.008518066788750472
0.060180721471218245
0.007004892727525385
0.1606633054199801
0.006931332637419013
0.006468561217041561
0.00240347365693076
0.004761652964968911
0.0030030133761535073
6.986192520644886e-08
-0.0003371964762489704
0.0012630240761361967
0.0009458361469521634
0.001853048950599914
0.0009180498353169443
0.000705461305021166
-0.0011161387564135943
0.00035641882216868556
0.0009215375697568626
-0.0017974392051610692
0.0017523282332521713
0.0005161360667011627
-0.0011922197424660017
0.00025082035957489114
0.00027637960602468973
0.00263145869038777

0.07555760453011287
0.09090632421680202
0.00032418583907556695
0.0001554750692799856
-0.020868934328326778
-0.017531012074156357
-0.012790254127439608
-0.011134436438755975
-0.015215313149886282
-0.01556802518826307
-0.023660274862137766
-0.019201843419443657
-0.005989190509525482
-0.017262402083191523
-0.00864686036179173
-0.00696496604274936
-0.00922570556034104
0.012431866786368994
-0.003712324659827051
0.09745082778066362
0.07186225803306186
0.09497618950386248
0.00382255868931612
0.010667176944717723
0.011897034736661712
0.0029457062197354475
0.002576912438747147
0.10402704435719277
0.11636900256333259
0.016973091342175606
0.03126697994829918
0.031001010366268178
0.008348456090538506
0.018162886700130765
0.022847701168437342
0.05501678389027921
0.3081379251688463
-0.005881367607047134
0.32578770713391586
0.07928268597724424
0.060009399097270044
0.21047169935453114
0.053996629076412776
0.021657548255218883
0.07247476707939224
0.06535817204343018
0.034983904464268134
0.0378982777920

In [16]:
print(drop_final31)

['V293', 'V295', 'V306', 'V308', 'V316', 'V318']
